In [4]:
from tsp_annealing import *
import matplotlib.pyplot as plt

In [5]:
opt_distance_51, opt_distance_280, opt_distance_442 = calculate_optimal_distances()

cities_51 = load_graph('TSP-Configurations/eil51.tsp.txt')
cities_280 = load_graph('TSP-Configurations/a280.tsp.txt')
cities_442 = load_graph('TSP-Configurations/pcb442.tsp.txt')

distances_51 = calculate_distances(cities_51)
distances_280 = calculate_distances(cities_280)
distances_442 = calculate_distances(cities_442)

In [34]:
def load_data_file(file_path):
    data = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            data.append(row)
    return data

def load_experiment(filepaths, labels):
    results = {}
    for (filepath, label) in zip(filepaths, labels):
        data = load_data_file(filepath)
        results[label] = data[2:]
        max_iter_list = data[1]
    return results, max_iter_list 

In [35]:
filepaths = [
    'generated_data/dist_perm_operator-swap_maxiter30001.csv', 
    'generated_data/dist_perm_operator-insert_maxiter30001.csv',
    'generated_data/dist_perm_operator-reverse_maxiter30001.csv'
    ]
labels = ['swap', 'insert', 'reverse']

data_perm_operators, max_iter_list = load_experiment(filepaths, labels)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [30]:
def plot_comparison(results, max_iterations_list, param_labels):
    plt.figure(figsize=(8, 6))
    colors = ['blue', 'red', 'green']

    opt_distance_line = [opt_distance_280 for _ in max_iterations_list]

    for i, param_label in enumerate(param_labels):
        means = np.array([np.mean(elem) for elem in results[param_label]])
        stds = np.array([np.std(elem) for elem in results[param_label]])
        z_score = 1.96 

        conf_intervals = np.column_stack([
        means - z_score * (stds / np.sqrt(len(means))),
        means + z_score * (stds / np.sqrt(len(means)))
        ])

        lower_bounds = np.array([interval[0] for interval in conf_intervals])
        upper_bounds = np.array([interval[1] for interval in conf_intervals])

        plt.plot(max_iterations_list, means, marker='o', linestyle='-', color=colors[i], label=param_label)
        plt.fill_between(max_iterations_list, lower_bounds, upper_bounds, alpha=0.3)
        
    plt.plot(max_iterations_list, opt_distance_line, linestyle='--', color='black', label='Optimal Distance')
    plt.title('Mean Distance over Number of Max Iterations')
    plt.xlabel('Max Iterations')
    plt.ylabel('Mean Distance')
    plt.legend()
    plt.show()



In [31]:
plot_comparison(data_perm_operators, max_iter_list, param_labels=labels)

[['2889.784830809678', '8422.616597322389', '6843.099133403052', '6146.479931669172', '5856.274135755367', '6366.6850072493335', '5538.594506102643'], ['2818.621642239474', '8356.7346642373', '6960.279827784866', '6517.19860171614', '5968.434791031504', '6285.531006137252', '5847.081957488106'], ['2993.226947715919', '8384.918865406084', '7797.4030885070215', '6624.6654421271105', '6562.892560394685', '6328.739823633847', '5375.96990171789'], ['3555.468445252576', '7945.973615063622', '6080.721659344486', '6365.615616824331', '5987.102872305898', '5452.810919034924', '5454.509080403629'], ['3203.0971410510465', '8744.968971930706', '5917.113960032042', '6789.907769959796', '5909.778386142694', '5846.6117241207485', '5156.968656446559'], ['3277.7106563050265', '8565.018035877865', '7909.373864699595', '6157.938764403495', '6490.965608543547', '6387.077322369351', '5564.995157140289'], ['2928.794043556948', '8352.394334618144', '6633.832248887842', '7220.052800432896', '6116.136446034338

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U18'), dtype('<U18')) -> None

<Figure size 800x600 with 0 Axes>